In [2]:
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt

# Impor untuk PCA dan variannya
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA

# Impor untuk Manifold Learning
from sklearn.manifold import LocallyLinearEmbedding, TSNE

# Impor dataset
from sklearn.datasets import fetch_openml, make_swiss_roll

# Impor untuk pipeline dan evaluasi
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

# Mengatur default plotting
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

print("--- BAB 8: PENGURANGAN DIMENSI (DIMENSIONALITY REDUCTION) ---")
print("Semua library telah diimpor.")

# 1. PRINCIPAL COMPONENT ANALYSIS (PCA)
print("\n[1. Principal Component Analysis (PCA)]")
# Teori: PCA adalah algoritma Proyeksi (Projection).
# Ia mengidentifikasi hyperplane yang paling dekat dengan data,
# lalu memproyeksikan data ke hyperplane tersebut[cite: 1124].
# Tujuannya adalah untuk mempertahankan 'variance' (keragaman) data sebanyak mungkin[cite: 1125].

# 1a. Konsep: PCA dengan SVD (Singular Value Decomposition)
print("  Menghitung PCA secara manual dengan SVD...")
# Membuat data 3D sintetis yang hampir datar (terletak di bidang 2D)
np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1
angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X_3d = np.empty((m, 3))
X_3d[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X_3d[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X_3d[:, 2] = X_3d[:, 0] * w1 + X_3d[:, 1] * w2 + noise * np.random.randn(m)

# Langkah 1: Pusatkan data (mean = 0)
X_3d_centered = X_3d - X_3d.mean(axis=0)

# Langkah 2: Gunakan SVD untuk mendapatkan semua Principal Components
U, s, Vt = np.linalg.svd(X_3d_centered)

# Principal Components (PC) adalah vektor W
# Vt adalah V transpose
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]
print(f"  PC 1: {c1}\n  PC 2: {c2}")

# Langkah 3: Proyeksikan data ke 2D (ke bidang yang dibentuk oleh 2 PC pertama)
W2 = Vt.T[:, :2]
X_2d_svd = X_3d_centered.dot(W2)

# 1b. PCA menggunakan Scikit-Learn
print("  Melakukan PCA dengan Scikit-Learn...")
# Scikit-Learn otomatis menangani pemusatan data (centering)
pca = PCA(n_components=2)
X_2d_sklearn = pca.fit_transform(X_3d)

# Membandingkan hasil SVD manual dan Scikit-Learn
# (Bisa terbalik, misal c1 = -c1_sklearn, tapi bidangnya sama)
print(f"  Komponen (PC) dari SK-Learn:\n{pca.components_.T}")

# 1c. Explained Variance Ratio (Rasio Varians yang Dijelaskan)
print("  Menghitung Explained Variance Ratio...")
# Teori: Ini memberi tahu kita berapa persen varians data
# yang terletak di sepanjang setiap Principal Component.
print(f"  Rasio Varians: {pca.explained_variance_ratio_}")
# Hasilnya menunjukkan bahwa PC1 menampung sebagian besar varians,
# PC2 menampung sisa varians (total ~100%), dan PC3 (yang tidak kita pilih)
# menampung hampir 0% varians.

# 1d. Memilih Jumlah Dimensi yang Tepat
print("  Memilih jumlah dimensi untuk MNIST...")
# Mari kita gunakan dataset MNIST
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
X_mnist = mnist["data"]
y_mnist = mnist["target"].astype(np.uint8)

X_train_mnist = X_mnist[:60000]
y_train_mnist = y_mnist[:60000]

# Latih PCA tanpa mengurangi dimensi untuk melihat semua varians
pca_mnist = PCA()
pca_mnist.fit(X_train_mnist)

# Hitung jumlah kumulatif dari varians
cumsum = np.cumsum(pca_mnist.explained_variance_ratio_)

# Cari jumlah dimensi yang diperlukan untuk 95% varians
d = np.argmax(cumsum >= 0.95) + 1
print(f"  Dimensi yang dibutuhkan untuk 95% varians: {d} (dari 784)")

# Cara lebih mudah: jalankan PCA dengan n_components=float (0.0 - 1.0)
pca_95 = PCA(n_components=0.95)
X_mnist_reduced = pca_95.fit_transform(X_train_mnist)
print(f"  Bentuk data setelah reduksi 95%: {X_mnist_reduced.shape}")

# 1e. PCA untuk Kompresi (dan Dekompresi)
print("  Menggunakan PCA untuk kompresi MNIST...")
# Setelah reduksi, kita bisa dekompresi kembali ke 784D
# Ini akan kehilangan sedikit informasi (5% varians),
# tapi harusnya masih mirip. Ini disebut 'reconstruction error'.
X_mnist_recovered = pca_95.inverse_transform(X_mnist_reduced)

# Fungsi helper untuk plot
def plot_digits(instances, images_per_row=5, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = mpl.cm.binary, **options)
    plt.axis("off")

# Plot perbandingan (di-comment)
# plt.figure(figsize=(7, 4))
# plt.subplot(121)
# plot_digits(X_train_mnist[::2100])
# plt.title("Asli", fontsize=16)
# plt.subplot(122)
# plot_digits(X_mnist_recovered[::2100])
# plt.title("Telah Dikompresi (95%)", fontsize=16)
# plt.show()

# 1f. Randomized PCA
# Scikit-Learn memiliki 'Randomized PCA' yang jauh lebih cepat
# Bekerja baik jika d (target dimensi) jauh lebih kecil dari n (fitur asli)
print("  Melatih Randomized PCA...")
rnd_pca = PCA(n_components=154, svd_solver="randomized", random_state=42)
X_mnist_reduced_rnd = rnd_pca.fit_transform(X_train_mnist)
# Hasilnya sama, tapi jauh lebih cepat.

# 1g. Incremental PCA (IPCA)
# Teori: Berguna untuk dataset besar (out-of-core) atau online learning.
# IPCA memecah data latih menjadi mini-batches dan melatihnya
# satu per satu menggunakan 'partial_fit()'.
print("  Melatih Incremental PCA (IPCA)...")
n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_train_mnist, n_batches):
    inc_pca.partial_fit(X_batch)

X_mnist_reduced_inc = inc_pca.transform(X_train_mnist)
# Hasilnya akan sangat mirip dengan PCA biasa.

# 2. KERNEL PCA (kPCA)
print("\n[2. Kernel PCA (kPCA)]")
# Teori: kPCA menggunakan 'Kernel Trick' (dari Bab 5)
# untuk melakukan reduksi dimensi non-linear yang kompleks.
# Berguna untuk 'membuka' manifold yang terpilin.

X_swiss, y_swiss_continuous = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)
# Convert continuous y_swiss to discrete labels for classification
y_swiss = np.array(y_swiss_continuous > np.median(y_swiss_continuous), dtype=int)


# Melatih kPCA dengan kernel RBF
rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04, random_state=42)
X_reduced_kpca = rbf_pca.fit_transform(X_swiss)

# Plot hasil kPCA (di-comment)
# plt.figure(figsize=(8, 6))
# plt.scatter(X_reduced_kpca[:, 0], X_reduced_kpca[:, 1], c=y_swiss, cmap=plt.cm.hot)
# plt.title("kPCA (RBF) membuka Swiss Roll")
# plt.xlabel("$z_1$")
# plt.ylabel("$z_2$")
# plt.grid(True)
# plt.show()

# 2a. Memilih Kernel dan Tuning Hyperparameter
print("  Tuning hyperparameter kPCA menggunakan GridSearchCV...")
# Teori: Karena kPCA adalah unsupervised, kita tidak bisa
# men-tuningnya berdasarkan 'akurasi'.
# TAPI, kita bisa membuat pipeline: 1. kPCA, 2. Klasifikasi.
# Lalu kita cari hyperparameter kPCA yang memberikan akurasi klasifikasi terbaik.

clf_kpca = Pipeline([
    ("kpca", KernelPCA(n_components=2)),
    ("log_reg", LogisticRegression(solver="lbfgs", max_iter=1000)) # Added max_iter for convergence
])

param_grid = [{
    "kpca__gamma": np.linspace(0.03, 0.05, 10),
    "kpca__kernel": ["rbf", "sigmoid"]
}]

grid_search = GridSearchCV(clf_kpca, param_grid, cv=3)
grid_search.fit(X_swiss, y_swiss)

print(f"  Hyperparameter kPCA terbaik (hasil GridSearchCV): {grid_search.best_params_}")

# 2b. kPCA: Rekonstruksi (Pre-image)
# Teori: Merekonstruksi data dari kPCA itu sulit.
# Kita bisa melatih 'inversi' aproksimatif.
print("  Melatih kPCA dengan dukungan inversi...")
rbf_pca_recon = KernelPCA(n_components=2, kernel="rbf", gamma=0.0433,
                          fit_inverse_transform=True, random_state=42)
X_reduced_recon = rbf_pca_recon.fit_transform(X_swiss)
X_preimage = rbf_pca_recon.inverse_transform(X_reduced_recon)

# Hitung reconstruction error
recon_error = mean_squared_error(X_swiss, X_preimage)
print(f"  Reconstruction Pre-image Error: {recon_error:.4f}")


# 3. LLE (LOCALLY LINEAR EMBEDDING)
print("\n[3. LLE (Locally Linear Embedding)]")
# Teori: LLE adalah algoritma Manifold Learning.
# LLE tidak memproyeksikan data, tetapi 'membuka' manifold.
# Ia bekerja dengan melihat hubungan linear lokal setiap instance
# dengan tetangga terdekatnya, lalu mencoba mempertahankan
# hubungan lokal tersebut di ruang berdimensi rendah.

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10, random_state=42)
X_reduced_lle = lle.fit_transform(X_swiss)

# Plot hasil LLE (di-comment)
# plt.figure(figsize=(8, 6))
# plt.scatter(X_reduced_lle[:, 0], X_reduced_lle[:, 1], c=y_swiss, cmap=plt.cm.hot)
# plt.title("LLE membuka Swiss Roll")
# plt.xlabel("$z_1$")
# plt.ylabel("$z_2$")
# plt.grid(True)
# plt.show()
# Hasilnya: Swiss roll terbuka, tapi skalanya sedikit terdistorsi.

# 4. TEKNIK LAINNYA (Disebutkan di buku)
print("\n[4. Teknik Lainnya (MDS, Isomap, t-SNE, LDA)]")
# - Multidimensional Scaling (MDS): Mencoba mempertahankan jarak antar instance.
# - Isomap: Menjaga 'geodesic distance' (jarak berjalan di atas manifold).
# - t-SNE: Sangat baik untuk visualisasi (terutama clustering),
#   ia menjaga instance serupa tetap dekat dan instance tidak serupa tetap jauh.
# - LDA (Linear Discriminant Analysis): Sebenarnya algoritma klasifikasi,
#   tapi ia belajar sumbu (axes) yang paling baik 'memisahkan' kelas,
#   yang bisa digunakan untuk reduksi dimensi.

print("\n--- Selesai Bab 8 ---")

--- BAB 8: PENGURANGAN DIMENSI (DIMENSIONALITY REDUCTION) ---
Semua library telah diimpor.

[1. Principal Component Analysis (PCA)]
  Menghitung PCA secara manual dengan SVD...
  PC 1: [0.93636116 0.29854881 0.18465208]
  PC 2: [-0.34027485  0.90119108  0.2684542 ]
  Melakukan PCA dengan Scikit-Learn...
  Komponen (PC) dari SK-Learn:
[[ 0.93636116 -0.34027485]
 [ 0.29854881  0.90119108]
 [ 0.18465208  0.2684542 ]]
  Menghitung Explained Variance Ratio...
  Rasio Varians: [0.84248607 0.14631839]
  Memilih jumlah dimensi untuk MNIST...
  Dimensi yang dibutuhkan untuk 95% varians: 154 (dari 784)
  Bentuk data setelah reduksi 95%: (60000, 154)
  Menggunakan PCA untuk kompresi MNIST...
  Melatih Randomized PCA...
  Melatih Incremental PCA (IPCA)...

[2. Kernel PCA (kPCA)]
  Tuning hyperparameter kPCA menggunakan GridSearchCV...
  Hyperparameter kPCA terbaik (hasil GridSearchCV): {'kpca__gamma': np.float64(0.05), 'kpca__kernel': 'rbf'}
  Melatih kPCA dengan dukungan inversi...
  Reconstructi